In [ ]:
import os

try:
    has_changed_dir
except:
    has_changed_dir = False

try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

if IN_COLAB:
    %pip install datasets
    %pip install translate-toolkit
    #%pip install bitsandbytes

    !git clone https://github.com/MartinKirkegaardDK/KDS_MI.git

    if not has_changed_dir:
        os.chdir('KDS_MI')
        has_changed_dir = True
else:
    if not has_changed_dir:
        os.chdir('.')
        has_changed_dir = True

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from drivers import (
    preprocessing,probe_confidence_intervals, 
    steering, create_steering_vectors, 
    steering_loss, 
    distance_plots, 
    create_average_activation_vectors, 
    visualising_steering_vectors,
    visualising_activations,
    visualize_probe_results,
    gen_steering_data_for_bible
)

from pathlib import Path


## Data preprocessing

Preprocesses the multilingual data, by removing short sentences

In [ ]:
preprocessing.run()

## Analysis of residual stream activations

### PCA of activations

In [ ]:
visualising_activations.run(model_name="EleutherAI/pythia-14m")

### Classification probes

In [ ]:
probe_confidence_intervals.run(
    model_name="AI-Sweden-Models/gpt-sw3-356m",
    reg_lambdas=[0.1, 0.5, 1, 2, 5, 10]
)

In [ ]:
result_folder = Path('results/data/probe_confidence_intervals')

visualize_probe_results.run(
    probe_result_path_by_reg_lambda={
        0.1: result_folder / 'AI-Sweden-Models-gpt-sw3-356m_reg_lambda_0.1.json',
        0.5: result_folder / 'AI-Sweden-Models-gpt-sw3-356m_reg_lambda_0.5.json',
        1: result_folder / 'AI-Sweden-Models-gpt-sw3-356m_reg_lambda_1.json',
        2: result_folder / 'AI-Sweden-Models-gpt-sw3-356m_reg_lambda_2.json',
        5: result_folder / 'AI-Sweden-Models-gpt-sw3-356m_reg_lambda_5.json',
        10: result_folder / 'AI-Sweden-Models-gpt-sw3-356m_reg_lambda_10.json',
    }
)



## plots the distance plots

In [ ]:
#AI-Sweden-Models/gpt-sw3-356m
distance_plots.run("da","average_activation_vectors/gpt_sw3_356m/","euclidean")

## Create average activation vectors
these are the "base" steering vectors e.i. the average activations for different layers and languages.
This function generates them and the next block then splits them up for the analysis section

In [ ]:
create_average_activation_vectors.run(["nb","is","da","en","sv"],"AI-Sweden-Models/gpt-sw3-356m","test_test")

## Create steering vector
these are the steering vectors towards a specific language. This is different that

In [ ]:
#AI-Sweden-Models/gpt-sw3-356m
create_steering_vectors.run(
    model_name="AI-Sweden-Models/gpt-sw3-356m",
    target_language="nb", 
    complement_languages=["is","da","en","sv"], 
    run_name="test_test"
)

## Test steering vectors

In [ ]:
steering_vector_path = "steering_vectors/DA_PYTHIA/combined_steering_vector_layer_4_tensor.pt"
model_name = 'EleutherAI/pythia-14m'

steering.run(
    steering_vector_path=steering_vector_path,
    steering_lambda=5,
    affected_language="en",
    layer=4,
    model_name=model_name,
    data_folder='data/preprocessed/train'
)

## Measure steering vector loss impact

In [ ]:
steering_vector_folder = 'steering_vectors/DA_PYTHIA'

steering_loss.run(
    steering_vector_folder=steering_vector_folder,
    model_name='EleutherAI/pythia-14m',
    steering_lambda=5
)

## Plot steering vector PCAs

In [ ]:
steering_vector_folder = Path('steering_vectors/')

visualising_steering_vectors.run(
    steering_vector_paths_by_language={
        'en': lambda layer: steering_vector_folder / f'EN/combined_steering_vector_layer_{layer}_tensor.pt',
        'da': lambda layer: steering_vector_folder / f'DA/combined_steering_vector_layer_{layer}_tensor.pt',
        'sv': lambda layer: steering_vector_folder / f'SV/combined_steering_vector_layer_{layer}_tensor.pt',
        'nb': lambda layer: steering_vector_folder / f'NB/combined_steering_vector_layer_{layer}_tensor.pt',
        'is': lambda layer: steering_vector_folder / f'IS/combined_steering_vector_layer_{layer}_tensor.pt'
    },
    hidden_layers=24
)

## Bible study

In [ ]:
steering_vector_path = "steering_vectors/test_run_2/"
model_name = "AI-Sweden-Models/gpt-sw3-356m"
language1 = "da"
langauge2 = "en"
start_verse = 3
gen_steering_data_for_bible.run(steering_vector_path,model_name, language1, langauge2,start_verse)